In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

'''
This script trains the cell segmentation U-Net

@author: jblugagne
Small augmentations: georgeoshardo
'''
# pip install "scikit_image==0.16.2"
import sys
sys.path.insert(1, '../SyMBac/')

from SyMBac.external.DeLTA.model import unet_seg
from SyMBac.external.DeLTA.data import trainGenerator_seg
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from natsort import natsorted
from tqdm.auto import tqdm

2023-05-10 01:38:53.803966: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
from glob import glob

In [3]:
data_dirs = list(reversed(natsorted(glob("data/*"))))

In [4]:
data_dirs

['data/6_12', 'data/5_10', 'data/4_8', 'data/3_6', 'data/2_4', 'data/1.5_3']

In [5]:
pbar = tqdm(data_dirs)
for data_dir in pbar:
    pbar.set_description("Processing %s" % data_dir)
    DeLTA_data =  f"{data_dir}/DeLTA/"
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=1)

    training_set = DeLTA_data
    model_file = DeLTA_data + 'models/test_new_saved-model_{epoch:02d}_{loss:.4f}.hdf5' #'/unet.hdf5'

    # Parameters:
    target_size = (832,832)
    input_size = target_size + (1,)
    batch_size = 4
    epochs = 100 #25
    steps_per_epoch = 40  #25

    #Data generator:
    data_gen_args = dict(
                        rotation = 0.5,
                        shiftX=.08,
                        shiftY=.08,
                        zoom=.17,
                        horizontal_flip=True,
                        histogram_voodoo=True,
                        illumination_voodoo=True)

    myGene = trainGenerator_seg(batch_size,
                               training_set + 'preprocessed/CROPPED_FILTERED/',
                               training_set + 'preprocessed/CROPPED_MASKS/',
                               training_set + 'preprocessed/WEIGHTMAPS/',
                               augment_params = data_gen_args,
                               target_size = target_size)


    # Define model:
    model = unet_seg(input_size = input_size)
    model.summary()
    model_checkpoint = ModelCheckpoint(model_file, monitor='loss',verbose=1, save_best_only=False, callback=callback)

    #filepath = "saved-model-{epoch:02d}-{loss:.2f}.hdf5"
    # Train it:
    model.fit_generator(myGene,steps_per_epoch=steps_per_epoch,epochs=epochs,callbacks=[model_checkpoint])

  0%|          | 0/6 [00:00<?, ?it/s]

2023-05-10 01:38:54.475955: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-05-10 01:38:54.478796: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-10 01:38:54.478868: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:06:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2023-05-10 01:38:54.478875: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-05-10 01:38:54.491424: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-05-10 01:38:54.491463: I tensorflow/stream_executor/pl

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
true_input (InputLayer)         [(None, 832, 832, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 832, 832, 64) 640         true_input[0][0]                 
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 832, 832, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 416, 416, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

2023-05-10 01:38:55.675049: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-05-10 01:38:55.675263: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2995200000 Hz


Epoch 1/100


2023-05-10 01:38:56.159038: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2023-05-10 01:38:57.249611: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8801
2023-05-10 01:38:57.970545: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-05-10 01:38:57.970570: W tensorflow/stream_executor/gpu/asm_compiler.cc:56] Couldn't invoke ptxas --version
2023-05-10 01:38:57.971021: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-05-10 01:38:57.971060: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-05-10 01:38:58.018437: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully ope

40/40 [==============================] - 36s 655ms/step - loss: 0.3302 - unstack_acc: 0.8959

Epoch 00001: saving model to data/6_12/DeLTA/models/test_new_saved-model_01_0.3302.hdf5
Epoch 2/100
40/40 [==============================] - 26s 662ms/step - loss: 0.1510 - unstack_acc: 0.9504

Epoch 00002: saving model to data/6_12/DeLTA/models/test_new_saved-model_02_0.1510.hdf5
Epoch 3/100
40/40 [==============================] - 27s 666ms/step - loss: 0.1262 - unstack_acc: 0.9587

Epoch 00003: saving model to data/6_12/DeLTA/models/test_new_saved-model_03_0.1262.hdf5
Epoch 4/100
40/40 [==============================] - 27s 666ms/step - loss: 0.1251 - unstack_acc: 0.9580

Epoch 00004: saving model to data/6_12/DeLTA/models/test_new_saved-model_04_0.1251.hdf5
Epoch 5/100
40/40 [==============================] - 27s 666ms/step - loss: 0.1029 - unstack_acc: 0.9659

Epoch 00005: saving model to data/6_12/DeLTA/models/test_new_saved-model_05_0.1029.hdf5
Epoch 6/100
40/40 [=======================

KeyboardInterrupt: 